In [ ]:
# 필요 라이브러리 설치
! pip install -U pandas-profiling[notebook]
! pip install category_encoders==2.*
! pip install gdown
! pip install eli5
! pip install shap

In [ ]:
import gdown # 대용량 구글드라이브 파일 다운로드를 위한 라이브러리
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import OrdinalEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from pandas_profiling import ProfileReport
from random import randint
import eli5
from eli5.sklearn import PermutationImportance
from xgboost import XGBClassifier
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# 데이터 불러오기

# 데이터 선정 이유: 일상생활 관련이 있는 데이터 분석 & 산업규모가 큰 도메인 (금융)

# 원본 데이터는 2015년 부터 2018년까지의 Lending Club에 대출을 신청한 사람 중 '승인'을 받은 건 (Row 갯수 1백만 개 이상)
# 데이터가 너무 많기 때문에, 가장 최근인 2018년의 데이터만 추출하여 CSV 파일로 구글드라이브에 저장

url = 'https://drive.google.com/uc?id=10yg9PH5V1hrFZEfxxh1ozl3VljQEJr7d'
output = 'lendingclub_accepted_2018.csv'
gdown.download(url, output, quiet=False)

In [ ]:
# 구글드라이브 csv 파일 불러오기
# 총 236,057개의 행이 존재
df_origin = pd.read_csv(output, encoding= 'unicode_escape')
df_origin.info()

In [ ]:
# 이 프로젝트의 목적은 대출인이 대출 시에 제출한 여러 특성이 대출 완납 여부에 어떻게 영향을 미치는지 알아보기 위함
# 이를 통해 대출인의 주요 특성만을 가지고 이 사람이 대출을 상환할 수 있는 사람인지를 예측하는 모델을 만드는 것이 목적

# 따라서 타겟 특성인 'loan_status'에 대해 먼저 살펴볼 필요가 있음

target = 'loan_status'
df_origin.loan_status.value_counts()

In [ ]:
# 총 5개의 상태값이 존재함

#상환 중이면 Current, 완납이면 Fully Paid, 연체 중이면 Late (31-120 days), 단기 연체 중이면 In Grace Period, 상환불가이면 Charged Off
#이 중 우리가 궁금한 것은 대출을 완벽히 상환했거나, 상환하지 못했거나 결론 내릴 수 있는 결과 값임
#따라서 현재 상환 중이거나, 연체 중인 경우는 명확히 결론 내릴 수 없기 때문에 제외하고, 'Fully Paid'와 'Charged Off' 두가지 값을 타겟 특성의 value로 설정

df = df_origin[df_origin.loan_status.isin(['Fully Paid', 'Charged Off'])]
df.info()
# 두가지만 남기니 32,834개의 행이 남음

In [ ]:
# 타겟값 설정을 위해 Fully Paid': 1, 'Charged Off': 0로 맵핑 진행
mapping = {'Fully Paid': 1, 'Charged Off': 0}
df['loan_status'] = df['loan_status'].map(mapping).astype(int)

In [ ]:
# 기준모델 설정을 위해 타겟의 비율 확인
df[target].value_counts(normalize=True)

In [ ]:
# 전액 상환 한 비율은 0.835536 따라서 이를 기준모델로 설정하고 예측을 진행
base_line = 0.835536

In [ ]:
# 데이터 전처리 진행

# 50을 초과하는 카디널리티를 가진 행 제거
selected_cols = df.select_dtypes(include=['number', 'object'])
colnames = selected_cols.columns.tolist()
labels = selected_cols.nunique()
    
selected_features = labels[labels <= 50].index.tolist()
df = df[selected_features]
df.head()

In [ ]:
# 여전히 너무 많은 행이 존재함
# 결측치가 과도하게 많은 행 제거 필요

# 결측치 갯수 확인
df.isnull().sum().sort_values(ascending=False).head(50)

In [ ]:
# 결측치가 3천개(약 10%) 이상 존재하는 행 제거
df = df.dropna(axis=1, thresh=3000)
df.head()

In [ ]:
# 하나의 값만 가지고 있는 행 삭제 (변별력 없음)
drop_cols = [c for c
             in list(df)
             if df[c].nunique() <= 1]
df = df.drop(columns=drop_cols)
df.head()

In [ ]:
# 데이터셋 요약해서 보기
df.profile_report()

In [ ]:
# profile_report를 통한 데이터 탐색을 기반으로 일부 특성 제거
drop_list = ['purpose', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'tax_liens', 'debt_settlement_flag',
             'inq_last_6mths', 'last_pymnt_d', 'last_credit_pull_d', 'grade']
df = df.drop(drop_list, axis=1)


# 삭제 이유
# purpose: title과 동일한 목적 (purpose 대출인이 작성한 대출목적, title 회사에서 입력한 대출목적)
# acc_now_delinq, delinq_amnt, num_tl_30dpd, num_tl_90g_dpd_24m, tax_liens, debt_settlement_flag: 특정 값이 절대 다수를 차지함 (95% 이상)
# chargeoff_within_12_mths: 타겟값과 연관성이 높은 Feature라서 제외
# inq_last_6mths, last_pymnt_d, last_credit_pull_d: 새로 대출이 발생했을 때 상환여부를 예측해야하는데, 최신 업데이트 값은 들어가서는 안됨
# grade: 더 상세하게 분류되어 있는 sub_grade가 있음

In [ ]:
# 특성 간의 상관관계 분석을 위해 피어슨 상관계수 분석
sns.set(style="whitegrid", font_scale=1)

plt.figure(figsize=(18,18))
plt.title('Pearson Correlation Matrix',fontsize=22)
sns.heatmap(df.corr(),linewidths=0.25,vmax=0.7,square=True,cmap="GnBu",linecolor='w',
            annot=True, annot_kws={"size":10}, cbar_kws={"shrink": .7})

In [ ]:
# 타겟값이 아님에도 불구하고 상관계수가 0.7이상인 값들은 둘 중 하나의 feature만 남기고 제거
high_corr = ['pub_rec', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_op_rev_tl', 'open_rv_12m', 'num_rev_tl_bal_gt_0', 'num_tl_op_past_12m', 'acc_open_past_24mths', 'fico_range_low']
df = df.drop(high_corr, axis=1)

In [ ]:
df.info()

In [ ]:
# train / test 셋 분리
train, test = train_test_split(df, train_size=0.80, test_size=0.20, 
                              stratify=df[target], random_state=2)

train.shape, test.shape

In [ ]:
# 타겟값 나누기
features = df.columns.drop(target)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
# Feature의 데이터 특성에 따라 적용할 인코더를 다르게 설정

#순서 없고 범주형인 Feature - TargetEncoder
tg_cat_cha = ['term', 'home_ownership', 'verification_status', 'issue_d', 'title', 'addr_state', 'initial_list_status', 'application_type', 'disbursement_method']

#순서 있고 범주형인 Feature 중에서 Mapping 지정할 것 - OrdinalEncoder
ord_cat_mapping_cha =[
                      {'col': 'sub_grade', 'mapping': {'A1': 35, 'A2': 34, 'A3': 33, 'A4': 32, 'A5': 31, 
                                                       'B1': 30, 'B2': 29, 'B3': 28, 'B4': 27, 'B5': 26,
                                                       'C1': 25, 'C2': 24, 'C3': 23, 'C4': 22, 'C5': 21,
                                                       'D1': 20, 'D2': 19, 'D3': 18, 'D4': 17, 'D5': 16,
                                                       'E1': 15, 'E2': 14, 'E3': 13, 'E4': 12, 'E5': 11,
                                                       'F1': 10, 'F2': 9, 'F3': 8, 'F4': 7, 'F5': 6,
                                                       'G1': 5, 'G2': 4, 'G3': 3, 'G4': 2, 'G5': 1}},
                      {'col': 'emp_length', 'mapping': {'< 1 year':0.5,
                                                        '1 year':1,
                                                        '2 years':2,
                                                        '3 years':3,
                                                        '4 years':4,
                                                        '5 years':5,
                                                        '6 years':6,
                                                        '7 years':7,
                                                        '8 years':8,
                                                        '9 years':9,
                                                        '10+ years':10}}
                                                        ]

In [ ]:
# 선형회귀, 로지스틱 회귀, 랜덤포레스트, XGBoost 총 4개의 모델을 가지고 기본적인 성능을 테스트

# linear regression 파이프라인 설정
linear = make_pipeline(
            TargetEncoder(cols = tg_cat_cha, smoothing = 400.0),
            OrdinalEncoder(mapping = ord_cat_mapping_cha, handle_missing = 'value'),
            SimpleImputer(strategy = 'mean'),
            StandardScaler(),
            LinearRegression()
            )

In [ ]:
# linear regression 모델 적용
linear.fit(X_train, y_train)

In [ ]:
print('linear regression 예측 정확도: ', linear.score(X_test, y_test))

In [ ]:
# Logistic regression 파이프라인 설정
logistic = make_pipeline(
            TargetEncoder(cols = tg_cat_cha, smoothing = 400.0),
            OrdinalEncoder(mapping = ord_cat_mapping_cha, handle_missing = 'value'),
            SimpleImputer(strategy = 'mean'),
            StandardScaler(),
            LogisticRegression(max_iter=1000)
            )

In [ ]:
# Logistic regression 모델 적용
logistic.fit(X_train, y_train)

In [ ]:
print('Logistic regression 예측 정확도: ', logistic.score(X_test, y_test))

In [ ]:
# RandomForest 파이프라인 설정
random_f = make_pipeline(
            TargetEncoder(cols = tg_cat_cha),
            OrdinalEncoder(mapping = ord_cat_mapping_cha, handle_missing = 'value'),
            SimpleImputer(strategy = 'mean'),
            RandomForestClassifier(random_state = 2, n_jobs = -1, oob_score=True)
            )

In [ ]:
# RandomForest 모델 적용
random_f.fit(X_train, y_train)

In [ ]:
print('RandomForest 예측 정확도: ', random_f.score(X_test, y_test))

In [ ]:
# XGBoost 파이프라인 설정
XGB = make_pipeline(
            TargetEncoder(cols = tg_cat_cha, smoothing = 400),
            OrdinalEncoder(mapping = ord_cat_mapping_cha, handle_missing = 'value'),
            SimpleImputer(strategy = 'mean'),
            XGBClassifier(random_state=2, n_jobs=-1, objective= 'binary:logistic', base_score=0.835536)
            )

In [ ]:
# XGBoost 모델 적용
XGB.fit(X_train, y_train)

In [ ]:
print('XGBoost 예측 정확도: ', XGB.score(X_test, y_test))

In [ ]:
from sklearn.metrics import roc_auc_score
class_index = 1
y_pred_proba = XGB.predict_proba(X_test)[:, class_index]
print(f'Test AUC for class Fully paid:')
print(roc_auc_score(y_test, y_pred_proba)) # 범위는 0-1, 수치는 높을 수록 좋음

In [ ]:
print('기준모델: ', base_line)

linear regression 예측 정확도:  0.090

Logistic regression 예측 정확도:  0.839

RandomForest 예측 정확도:  0.839 (AUC score 0.737)

XGBoost 예측 정확도: 0.840

기준모델:  0.835536

분류 모델이기 때문에 선형회귀분석의 정확도가 가장 낮게 나온 것은 예상했던 결과

그러나 가장 높은 정확도를 예상했던 XGBoost가 로지스틱 회귀와 비슷하게

XGBoost의 하이퍼파라미터 값을 조정해서 가장 높은 정확도를 얻는 것이 목표


In [ ]:
# 최적의 K 찾기 - 교차 검증
k = list(range(3, 8))
for i in k :
        scores = cross_val_score(XGB, X_train, y_train, cv=i, 
        scoring = 'accuracy')
        print(f'Accuracy', scores)
        print(f'Accuracy mean', scores.mean())

# K가 6일 때 좋은 결과를 보임

In [ ]:
'''
# 최적의 하이퍼 파라미터 찾기
dists = {
    'targetencoder__smoothing': [100.,150.,300.,500.],  
    'xgbclassifier__n_estimators': [100, 300, 500, 1000], 
    'xgbclassifier__max_depth': [5, 10, 15, 20], 
    'xgbclassifier__min_child_weight':range(1,6,2), # 과적합(overfitting)을 방지할 목적으로 사용
    'xgbclassifier__gamma':[i/10.0 for i in range(0,5)],
    'xgbclassifier__subsample':[i/10.0 for i in range(6,10)], # 개별 의사결정나무 모형에 사용되는 임의 표본수를 지정
    'xgbclassifier__colsample_bytree':[i/10.0 for i in range(6,10)], # 개별 의사결정나무 모형에 사용될 변수갯수를 지정
    'xgbclassifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3]
}

# n_iter(=10) * 6 교차검증 = 60 tasks 수행
clf = RandomizedSearchCV(
    XGB, 
    param_distributions=dists,
    n_iter=10, 
    cv=6, 
    scoring='accuracy',  
    verbose=1
)

clf.fit(X_train, y_train);
print('최적 하이퍼 파라미터: ', clf.best_params_)
print('Accuracy: ', clf.best_score_)
'''

최적 하이퍼 파라미터: 

'xgbclassifier__subsample': 0.9

'xgbclassifier__n_estimators': 1000

'xgbclassifier__min_child_weight': 3

'xgbclassifier__max_depth': 5

'xgbclassifier__learning_rate': 0.01

'xgbclassifier__gamma': 0.0

'xgbclassifier__colsample_bytree': 0.8

'targetencoder__smoothing': 150.0


Accuracy:  0.8385426728633508

In [ ]:
# 위의 최적 파라미터 값을 대입하여 XGBoost 파이프라인 설정
XGB_final = make_pipeline(
            TargetEncoder(cols = tg_cat_cha, smoothing=150),
            OrdinalEncoder(mapping = ord_cat_mapping_cha, handle_missing = 'value'),
            SimpleImputer(strategy = 'mean'),
            XGBClassifier(random_state=2, n_jobs=-1, objective= 'binary:logistic', base_score=0.835536, n_estimators=1000,
                          subsample=0.9,min_child_weight=3, max_depth=5,learning_rate=0.01, gamma=0.0, colsample_bytree=0.8)
            )

In [ ]:
# XGBoost 모델 적용 (하이퍼파라미터 세팅)
XGB_final.fit(X_train, y_train)

In [ ]:
print('XGBoost 예측 정확도: ', XGB_final.score(X_test, y_test))

In [ ]:
from sklearn.metrics import roc_auc_score
class_index = 1
y_pred_proba = XGB_final.predict_proba(X_test)[:, class_index]
print(f'Test AUC for class Fully paid:')
print(roc_auc_score(y_test, y_pred_proba)) # 범위는 0-1, 수치는 높을 수록 좋음

XGBoost 예측 정확도  0.841, AUC Score가 0.739로 하이퍼 파라미터 세팅 후 모두 향상 됨

In [ ]:
# Confution matrix를 확인
from sklearn.metrics import classification_report
y_test_pred = XGB_final.predict(X_test)
print(classification_report(y_test, y_test_pred))

In [ ]:
# Confution matrix를 확인

from sklearn.metrics import classification_report
y_test_pred = XGB_final.predict(X_test)
print(classification_report(y_test, y_test_pred))

from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(XGB_final, X_test, y_test,
                            cmap=plt.cm.Blues,
                            ax=ax, values_format='');
plt.title(f'Confusion matrix, n = {len(y_test)}', fontsize=15)
plt.show()

상환할 것으로 예상했지만 실제로 부실인 경우가 굉장히 높게 나타남

In [ ]:
tp = 5450
tn = 74
fp = 1006
fn = 37
total = tp + tn + fp + fn

positives = tp + fp
print('정밀도: ', tp/positives)

real_positives = tp + fn
print('재현율: ', tp/real_positives)